# 本チャプターの目次
1. ビジネスメタデータの重要な役割
2. ビジネスメタデータの活用方法
3. 代表的なビジネスメタデータは？
4. 実際にビジネスメタデータを覗いてみよう
5. メタデータ保存用のテーブルを作成してみよう

# ビジネスメタデータの重要な役割

データマネジメントとは、

# ビジネスメタデータの活用方法


# 代表的なビジネスメタデータは？

# 実際にビジネスメタデータを覗いてみよう

# メタデータ保存用のテーブルを作成してみよう

In [ ]:
# コンソールで設定したSparkとNoteBookを接続します(動かす前に毎度実行する必要があります)
import findspark
findspark.init()

In [ ]:
#pysparkに必要なライブラリを読み込む
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql import SparkSession

#spark sessionの作成
# spark.ui.enabled trueとするとSparkのGUI画面を確認することができます
# spark.eventLog.enabled true　とすると　GUIで実行ログを確認することができます
# GUIなどの確認は最後のセクションで説明を行います。
spark = SparkSession.builder \
    .appName("chapter1") \
    .config("hive.exec.dynamic.partition", "true") \
    .config("hive.exec.dynamic.partition.mode", "nonstrict") \
    .config("spark.sql.session.timeZone", "JST") \
    .config("spark.ui.enabled","true") \
    .config("spark.eventLog.enabled","true") \
    .enableHiveSupport() \
    .getOrCreate()


# spark.xxxxxと記載することで処理を分散させることが可能です。